In [4]:
import dpkt
import matplotlib.pyplot as plt
import numpy
from scipy.spatial import distance as dist
import math
import pickle
import numpy as np
from scipy import spatial
from scipy import stats
import matplotlib
import os
from datetime import datetime
import json
import csv
from pathlib import Path

In [5]:
TransitionPeriod = 0.5
LENGTH = 14400

In [6]:
LENGTH = 3600
HOUR_secs = 3600
download_speed = 30000000
upload_speed = 25000000
DELTA = 15
delay_scales = [0.01, 0.05, 0.1, 0.2, 0.3, 0.5, 0.7, 1, 5]
delay_scales = [0.5, 1.0, 5]

In [ ]:
def get_message_file_names(b, e, messages_dir):
    all_messages_file_names = sorted(os.listdir(all_messages_path))
    if not os.path.exists(messages_dir):
        os.makedirs(messages_dir)
    for i in range(len(all_messages_file_names)):
        if i >= b and i <= e:
            print(all_messages_file_names[i])
            os.system('cp {} {}'.format(os.path.join(all_messages_path, all_messages_file_names[i]), os.path.join(messages_dir,all_messages_file_names[i])))
    return

all_messages_path = 'path/to/generated_messages/messages_pareto_xm_5000_alpha_0.93_max30delay_ver02'
root_dir = 'path/to/traces/directory'
timestamps_dirs = []
messages_dirs = []
message_file_names = []
packets_pickle_dirs = []
packets = []
dir_count = 0
for path in Path(root_dir).iterdir():
    if path.is_dir() and path.name != 'aggregate' and path.name != 'old':
        print(path)
        timestamps_dirs.append(os.path.join(path, 'timestamps'))
        messages_dirs.append(os.path.join(path, 'messages'))
        range_begin = int(path.name.split('_')[0])
        range_end = int(path.name.split('_')[1])
        print(range_begin, range_end)
        if len(os.listdir(all_messages_path)) == 0:
            get_message_file_names(range_begin, range_end, messages_dirs[dir_count])
        message_file_names.extend(sorted(os.listdir(messages_dirs[dir_count])))
        packets_pickle_dirs.append(os.path.join(path, 'traces/pickles/packets'))
        # read all packets from packets pickle and write them to 'packets'
        pickle_name = os.listdir(packets_pickle_dirs[dir_count])[0]
        print(pickle_name)
        with open(os.path.join(packets_pickle_dirs[dir_count], pickle_name[:-7] + '.pickle'), 'rb') as handle:
            packets.extend(pickle.load(handle))
            print(packets[-1])
        dir_count += 1
aggregate_dir = Path(root_dir) / 'aggregate'
aggregate_dir.mkdir(parents=True, exist_ok=True)
results_dir = (aggregate_dir / 'countermeasures')
results_dir.mkdir(parents=True, exist_ok=True)
results_dir = str(results_dir)

In [6]:
def get_sizes_sender(messages_file, max_count=None):
    sizes = []
    with open(messages_file) as f:
        data = json.load(f)
        count = 0
        for message in data['messages']:
            if max_count is None or count < max_count:
                sizes.append(int(message['size'])) # sizes are in bytes
            else:
                break
            count += 1
    return sizes

def get_timestamps_sender(timestamps_file, max_count=None):
    timestamps = []
    with open(timestamps_file) as f:
        lines = f.readlines()
        count = 0
        for line in lines:
            if max_count is None or count < max_count:
                line = line.split()
                timestamps.append((float)(line[2]))
            else:
                break
            count += 1
    return timestamps

def get_sizes_of_all_senders(messages_dir):
    message_file_names = sorted(os.listdir(messages_dir))
    all_message_sizes_of_senders = []
    for mf_name in message_file_names:
        message_sizes = get_sizes_sender(os.path.join(messages_dir, mf_name))
        all_message_sizes_of_senders.append(message_sizes)
    return all_message_sizes_of_senders

def get_timestamps_of_all_senders(messages_dir, timestamps_root_dir):
    message_file_names = sorted(os.listdir(messages_dir))
    all_timestamps = []
    for mf_name in message_file_names:
        timestamps = get_timestamps_sender(os.path.join(timestamps_root_dir, 'timestamps_' + mf_name[:-5] + '.txt'))
        all_timestamps.append(timestamps)
    return all_timestamps

def get_types_of_all_senders(messages_dir, max_count=None):
    message_file_names = sorted(os.listdir(messages_dir))
    all_types = []
    for messages_file in message_file_names:
        with open(os.path.join(messages_dir, messages_file)) as f:
            types = []
            data = json.load(f)
            count = 0
            for message in data['messages']:
                if max_count is None or count < max_count:
                    types.append((message['type']))
                else:
                    break
                count += 1
            all_types.append(types)
    return all_types

In [ ]:
timestamps_of_all_senders = []
message_sizes_of_all_senders = []
message_types_of_all_senders = []
for i in range(dir_count):
    timestamps_of_all_senders.extend(get_timestamps_of_all_senders(messages_dirs[i], timestamps_dirs[i]))
    message_sizes_of_all_senders.extend(get_sizes_of_all_senders(messages_dirs[i]))
    message_types_of_all_senders.extend(get_types_of_all_senders(messages_dirs[i]))

In [8]:
def separate_receiver_packets(packets, timestamps_of_all_senders):
    packets_of_all_receivers = []
    all_beginnings = []
    all_endings = []
    for sender_timestamps in timestamps_of_all_senders:
        beginning = sender_timestamps[0] - 5 #seconds
        ending = sender_timestamps[len(sender_timestamps)-1] + 30 #seconds
        all_beginnings.append(beginning)
        all_endings.append(ending)
    print(all_beginnings)
    print(all_endings)
    current_sender = 0
    num_of_senders = len(timestamps_of_all_senders)
    current_sender_received_packets = []
    for p in packets:
        if p.get('timestamp') > all_beginnings[current_sender] and p.get('timestamp') < all_endings[current_sender]:
            current_sender_received_packets.append(p)
        elif p.get('timestamp') >= all_endings[current_sender]:
            packets_of_all_receivers.append(current_sender_received_packets)
            if current_sender < num_of_senders - 1:
                current_sender += 1
                current_sender_received_packets = []
                if p.get('timestamp') > all_beginnings[current_sender] and p.get('timestamp') < all_endings[current_sender]:
                    current_sender_received_packets.append(p)
            else:
                break
    return packets_of_all_receivers

In [ ]:
packets_of_all_receivers = separate_receiver_packets(packets, timestamps_of_all_senders)
packets_towards_receivers = [[p for p in packets if (p.get('to_amazon') is False)] for packets in packets_of_all_receivers]

In [10]:
def convert_time(timestamps, start_time):
    converted_timestamps = []
    for t in timestamps:
        converted = t - start_time
        if converted < 0:
            print('Error: timestamp is larger than start time with t = {} and start_time = {}'.format(t, start_time))
        converted_timestamps.append(converted)
    return converted_timestamps    

def get_on_periods_of_sender_2(timestamps, message_sizes):
    global TransitionPeriod
    bursts = []
    for t, s in zip(timestamps, message_sizes):
        bursts.append((t, s))
    return bursts

def get_on_periods_of_user(timestamps, user_packet_sizes):
    global TransitionPeriod
    is_in_burst = False
    bursts = []
    burst_size = 0
    last_burst_index = 0
    for time_iter in range(0, len(timestamps)):
        if time_iter == len(timestamps) - 1 and is_in_burst == True:
            if burst_size > 1514:
                bursts.append((timestamps[last_burst_index], burst_size))
                break
        if timestamps[time_iter] - timestamps[last_burst_index] > TransitionPeriod:
            if is_in_burst == True:
                if burst_size > 1514:
                    bursts.append((timestamps[last_burst_index], burst_size))
                is_in_burst = False
                burst_size = 0
        if user_packet_sizes[time_iter] > 1400:
            if is_in_burst == False:
                is_in_burst = True
                last_burst_index = time_iter
            burst_size += user_packet_sizes[time_iter]
    return bursts

def get_bursts_user(timestamps, user_packet_sizes):
    global TransitionPeriod
    is_in_burst = False
    bursts = []
    burst_size = 0
    last_burst_index = 0
    first_burst_index = 0
    for time_iter in range(0, len(timestamps)):
        if time_iter == len(timestamps) - 1 and is_in_burst == True:
            if burst_size > 2000:
                bursts.append((timestamps[first_burst_index], timestamps[last_burst_index], burst_size))
                break
        if timestamps[time_iter] - timestamps[last_burst_index] > 0.5:
            if is_in_burst == True:
                if burst_size > 2000:
                    bursts.append((timestamps[first_burst_index], timestamps[last_burst_index], burst_size))
                is_in_burst = False
                burst_size = 0
        if user_packet_sizes[time_iter] > 1400:
            if is_in_burst == False:
                is_in_burst = True
                first_burst_index = time_iter
            last_burst_index = time_iter
            burst_size += user_packet_sizes[time_iter]
    return bursts

def get_bursts_user_2(timestamps, user_packet_sizes):
    bursts = []
    burst_size = 0
    for time_iter in range(0, len(timestamps) - 1):
        ipd = timestamps[time_iter + 1] - timestamps[time_iter]
        if ipd < 1:
            if user_packet_sizes[time_iter] > 40:
                burst_size += user_packet_size[time_iter]
        else:
            if burst_size > 20000:
                bursts.append((timestamps[time_iter], burst_size))
            burst_size = 0
    return bursts

def merge_channel_period_points(ChannelPeriodPoints):
    c = []
    time = [x[0] for x in ChannelPeriodPoints]
    size = [x[1] for x in ChannelPeriodPoints]
    burst_time = []
    burst_volume = []
    burst_volume.append(size[0])
    burst_time.append(time[0])
    temp = time[0]
    for i in range(len(time)-1):
        if time[i+1] - temp < 2:
            burst_volume[-1] += size[i+1]
            temp = time[i+1]
        else:
            burst_time.append(time[i+1])
            burst_volume.append(size[i+1])
            temp = time[i+1]
    for i in range(len(burst_time)):
        c.append((burst_time[i], burst_volume[i]))
    return c

In [11]:
def find_intervals(period_points, minute):
    global LENGTH
    num_of_intervals = int(LENGTH / minute)
    intervals = []
    for i in range(num_of_intervals):
        intervals.append([])
        startpoint = i * minute
        endpoint = (i + 1) * minute
        for j in range(len(period_points)):
            if startpoint <= period_points[j][0] and period_points[j][0] < endpoint:
                intervals[-1].append(period_points[j])
    return intervals

def find_intervals_channel(period_points, minute, message_types):
    global LENGTH
    num_of_intervals = int(LENGTH / minute)
    intervals = []
    for i in range(num_of_intervals):
        intervals.append([])
        startpoint = i * minute
        endpoint = (i + 1) * minute
        for j in range(len(period_points)):
            if startpoint <= period_points[j][0] and period_points[j][0] < endpoint and message_types[j] != 'text':
                intervals[-1].append(period_points[j])
    return intervals

def find_matches(channel_interval, user_interval):
    number_of_matches = 0
    number_of_nonmatches = 0
    matched_user_intervals = set()
    for j in range(min(len(channel_interval), len(user_interval))):
        time = channel_interval[j][0]
        size = channel_interval[j][1]
        is_matched = False
        for d in range(1, DELTA):
            for i in range(len(user_interval)):
    #             print (time, user_interval[i][0], DELTA)
                if user_interval[i] in matched_user_intervals:
                    continue
                if abs(user_interval[i][0] - time) < d:
                    if abs(size - user_interval[i][1]) < 0.25 * size:
                        matched_user_intervals.add(user_interval[i])
                        number_of_matches += 1
                        is_matched = True
                        break
            else:
                continue
            break
        if is_matched is False:
            number_of_nonmatches += 1
    return number_of_matches / float(number_of_matches + number_of_nonmatches)

def detect_correlations_from_bursts(channel_bursts, user_bursts, message_types, j):
    global observation_lengths
    correlations = []
    user_intervals = find_intervals(user_bursts, observation_lengths[j])
    channel_intervals = find_intervals_channel(channel_bursts, observation_lengths[j], message_types)
    correlation_of_intervals = detect_correlations_from_intervals(channel_intervals, user_intervals)
    return correlation_of_intervals

def detect_correlations_from_intervals(channel_intervals, user_intervals):
    correlation = []
    for i in range(len(channel_intervals)):
        if len(channel_intervals[i]) == 0 and len(user_intervals[i]) == 0: # no events in either one
            correlation.append(-1)
        elif len(channel_intervals[i]) == 0 or len(user_intervals[i]) == 0: # only one has an event
            correlation.append(0)
        else:
            correlation.append(find_matches(channel_intervals[i], user_intervals[i]))
    return correlation

def remove_empty_correlations(correlations):
    organized = []
    for i in range(len(correlations)):
        organized.append([c for c in correlations[i] if c != -1])
    return organized

## Delay

In [26]:
def delay(timestamps, scale):
    exp_delay = np.random.exponential(scale, size = len(timestamps))
    delay_sum = np.sum(exp_delay)
    for i in range(len(exp_delay)-1, -1, -1):
        timestamps[i] += delay_sum
        delay_sum -= exp_delay[i]
    return timestamps

In [27]:
def get_relevant_bursts(bursts, mode):
    new_bursts = []
    for i in range(len(bursts)):
        bursts_temp = []
        for b in bursts[i]:
            if mode == 'receiver':
                bursts_temp.append((b[1], b[2]))
            if mode == 'sender':
                bursts_temp.append((b[0], b[2]))
        new_bursts.append(bursts_temp)
    return new_bursts

def get_all_bursts_delayed(scale):
    all_user_bursts = []
    all_channel_bursts = []
    print('scale is {}'.format(scale))
    for i in range(0, len(packets_towards_receivers)):
        print('Hour {} {}'.format(i, message_file_names[i]))

        channel_timestamps = timestamps_of_all_senders[i]
        channel_message_sizes = message_sizes_of_all_senders[i]


        user_packet_timestamps = [p.get('timestamp') for p in packets_towards_receivers[i]]
        user_packet_sizes = [p.get('size') for p in packets_towards_receivers[i]]

        if len(user_packet_timestamps) == 0:
            all_user_bursts.append([])
            all_channel_bursts.append([])
            print('ERROR')
            continue
        start_time = min(channel_timestamps[0], user_packet_timestamps[0])

        converted_sender_timestamps = convert_time(channel_timestamps, start_time)

        sender_upload_times = [size / upload_speed for size in channel_message_sizes]
        sender_end_upload_timestamps = [t + d for t,d in zip(converted_sender_timestamps, sender_upload_times)]
        sender_period_points = list(tuple(zip(converted_sender_timestamps, sender_end_upload_timestamps, channel_message_sizes)))

        converted_user_timestamps = convert_time(user_packet_timestamps, start_time)
        delayed_converted_receiver_timestamps = delay(converted_user_timestamps, scale)
        user_period_points = get_bursts_user(converted_user_timestamps, user_packet_sizes)

        all_user_bursts.append(user_period_points)
        all_channel_bursts.append(sender_period_points)
        
        print ('Hour {} is done'.format(i))
        print ("\n")
        

    return all_channel_bursts, all_user_bursts

In [51]:
def get_tp_fp_delayed_for_scale(scale):
    all_channel_bursts, all_user_bursts = get_all_bursts_delayed(scale)
    with open(Path(results_dir) / 'delayed' / 'pickles' / 'all_user_bursts_delta_{}_delayed_{:.2f}.pickle'.format(DELTA, scale), 'wb') as handle:
        pickle.dump(all_user_bursts, handle, protocol=pickle.HIGHEST_PROTOCOL)
    with open(Path(results_dir) / 'delayed' / 'pickles' / 'all_channel_bursts_delta_{}_delayed_{:.2f}.pickle'.format(DELTA, scale), 'wb') as handle:
        pickle.dump(all_channel_bursts, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    all_channel_bursts = get_relevant_bursts(all_channel_bursts, 'sender')
    all_user_bursts = get_relevant_bursts(all_user_bursts, 'receiver')
    
    
    all_correlations_tp = []
#     for j in range(len(observation_lengths)):
    j = 3
    print('For {}-second interval:'.format(observation_lengths[j]))
    corrs_tp = []
    for i in range(len(all_user_bursts)): # for every hour
        corrs = detect_correlations_from_bursts(all_channel_bursts[i], all_user_bursts[i],
                                                message_types_of_all_senders[i], j)
        if corrs != -1:
            corrs_tp.extend(corrs)
        print ("Hour {} is done".format(i))
    all_correlations_tp.append([c for c in corrs_tp if c != -1])
    with open(Path(results_dir) / 'delayed'/ 'pickles' / 'corrs_tp_event_based_delta_{}_delay_{:.2f}.pickle'.format(DELTA, scale), 'wb') as handle:
        pickle.dump(all_correlations_tp, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    
    all_corrs_fp = []
#     for m in range(len(observation_lengths)):
    m = 3
    print('For {}-second interval:'.format(observation_lengths[m]))
    corrs_fp = []
    for i in range(len(all_user_bursts)):
        for j in range(len(all_channel_bursts)):
            if i == j:
                continue
            corrs = detect_correlations_from_bursts(all_channel_bursts[j], all_user_bursts[i], message_types_of_all_senders[j], m)
            if len(corrs) == 0:
                print('warning')
            if corrs != -1:
                corrs_fp.extend(corrs)
            print ("user {} with channel {} is done".format(i, j))
    all_corrs_fp.append([c for c in corrs_fp if c != -1])
    with open(Path(results_dir) / 'delayed'/ 'pickles' /'corrs_fp_event_based_delta_{}_delay_{:.2f}.pickle'.format(DELTA, scale), 'wb') as handle:
        pickle.dump(all_corrs_fp, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print('scale {} is done'.format(scale))

In [ ]:
get_tp_fp_delayed_for_scale(0.1)

## Padding

In [89]:
photos_dir = 'path/to/generated/media/photos'

with open(Path(results_dir) / 'padding'/ 'pickles' / 'photo_sizes.pickle', 'rb') as handle:
    photo_sizes = pickle.load(handle)

In [90]:
def pad_bursts(bursts, padding_ratio):
    padded_bursts = []
    for b in bursts:
        random_ratio = np.random.rand()
        added_size = b[2] * random_ratio * padding_ratio
        added_time = added_size  / download_speed
        padded_bursts.append((b[0], b[1] + added_time, b[2] + added_size))
    return sorted(padded_bursts, key = lambda tup: tup[1])

def generate_dummy_bursts(prob = 0.002):
    dummy_bursts = []
    timestamp = 0
    while timestamp < 3600:
        r = np.random.rand()
        if r < prob:
            random_size = np.random.choice(photo_sizes)
            added_time = random_size / download_speed
            if timestamp + added_time >= 3600:
                break
            else:
                dummy_bursts.append((timestamp, timestamp + added_time, random_size))
                timestamp += added_time
        else:
            s += 1
    return dummy_bursts

def add_padding_and_dummy_packets(bursts, padding_ratio, prob):
    combined_bursts = pad_bursts(bursts, padding_ratio)
    combined_bursts.extend(generate_dummy_bursts(prob))
    combined_bursts = sorted(combined_bursts, key = lambda tup: tup[1])
    return combined_bursts

In [91]:
def get_relevant_bursts(bursts, mode):
    new_bursts = []
    for i in range(len(bursts)):
        bursts_temp = []
        for b in bursts[i]:
            if mode == 'receiver':
                bursts_temp.append((b[1], b[2]))
            if mode == 'sender':
                bursts_temp.append((b[0], b[2]))
        new_bursts.append(bursts_temp)
    return new_bursts

def get_all_bursts_padded(padding_ratio, dummy_prob):
    all_user_bursts = []
    all_channel_bursts = []
    print('Padding ratio is {}'.format(padding_ratio))
    for i in range(0, len(packets_towards_receivers)):

        print('Hour {} {}'.format(i, message_file_names[i]))

        channel_timestamps = timestamps_of_all_senders[i]
        channel_message_sizes = message_sizes_of_all_senders[i]


        user_packet_timestamps = [p.get('timestamp') for p in packets_towards_receivers[i]]
        user_packet_sizes = [p.get('size') for p in packets_towards_receivers[i]]

        if len(user_packet_timestamps) == 0:
            all_user_bursts.append([])
            all_channel_bursts.append([])
            print('ERROR')
            continue
        start_time = min(channel_timestamps[0], user_packet_timestamps[0])

        converted_sender_timestamps = convert_time(channel_timestamps, start_time)

        sender_upload_times = [size / upload_speed for size in channel_message_sizes]
        sender_end_upload_timestamps = [t + d for t,d in zip(converted_sender_timestamps, sender_upload_times)]
        sender_period_points = list(tuple(zip(converted_sender_timestamps, sender_end_upload_timestamps, channel_message_sizes)))

        converted_user_timestamps = convert_time(user_packet_timestamps, start_time)
        user_period_points = get_bursts_user(converted_user_timestamps, user_packet_sizes)
        padded_user_period_points = pad_bursts(user_period_points, padding_ratio)


        all_user_bursts.append(padded_user_period_points)
        all_channel_bursts.append(sender_period_points)
        print ('Hour {} is done'.format(i))
        print ("\n")
        
    return all_channel_bursts, all_user_bursts

In [99]:
def get_tp_fp_with_padding(padding_ratio, dummy_prob):
    all_channel_bursts, all_user_bursts = get_all_bursts_padded(padding_ratio, dummy_prob)
    all_channel_bursts = get_relevant_bursts(all_channel_bursts, 'sender')
    all_user_bursts = get_relevant_bursts(all_user_bursts, 'receiver')
    
    
    all_correlations_tp = []
    for j in range(len(observation_lengths)):
        print('For {}-second interval:'.format(observation_lengths[j]))
        corrs_tp = []
        for i in range(len(all_user_bursts)): # for every hour
            corrs = detect_correlations_from_bursts(all_channel_bursts[i], all_user_bursts[i],
                                                    message_types_of_all_senders[i], j)
            if corrs != -1:
                corrs_tp.extend(corrs)
            print ("Hour {} is done".format(i))
        all_correlations_tp.append([c for c in corrs_tp if c != -1])
    with open(Path(results_dir) / 'padding'/ 'pickles' / 'corrs_tp_event_based_delta_{}_padding_ratio_{:.2f}.pickle'.format(DELTA, padding_ratio), 'wb') as handle:
        pickle.dump(all_correlations_tp, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
    
    all_corrs_fp = []
    for m in range(len(observation_lengths)):
        print('For {}-second interval:'.format(observation_lengths[m]))
        corrs_fp = []
        for i in range(len(all_user_bursts)):
            for j in range(len(all_channel_bursts)):
                if i == j:
                    continue
                corrs = detect_correlations_from_bursts(all_channel_bursts[j], all_user_bursts[i], message_types_of_all_senders[j], m)
                if len(corrs) == 0:
                    print('warning')
                if corrs != -1:
                    corrs_fp.extend(corrs)
                print ("user {} with channel {} is done".format(i, j))
        all_corrs_fp.append([c for c in corrs_fp if c != -1])
    with open(Path(results_dir) / 'padding'/ 'pickles' / 'corrs_fp_event_based_delta_{}_padding_ratio_{:.2f}.pickle'.format(DELTA, padding_ratio), 'wb') as handle:
        pickle.dump(all_corrs_fp, handle, protocol=pickle.HIGHEST_PROTOCOL)
    print('Padding ratio {} is done'.format(padding_ratio))

In [ ]:
get_tp_fp_with_padding(1, 0.002)